<a href="https://colab.research.google.com/github/fnilvuwu/facial-age-estimation-website/blob/main/CNN_Age_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from google.colab.patches import cv2_imshow
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Reshape, LSTM, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG19

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xf "/content/drive/MyDrive/UTKFace.tar.gz"

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Set path to UTKFace dataset
data_path = '/content/UTKFace'

# Load the list of filenames
filenames = os.listdir(data_path)

# Split the filenames into training and validation sets
train_filenames, val_filenames = train_test_split(filenames, test_size=0.2, random_state=42)

# Define function to load and preprocess images
def load_image(filename):
    if (len(filename.split('_')) == 4):
        age, _, _, _ = filename.split('_')
    else:
        age, _, _ = filename.split('_')
    img = cv2.imread(os.path.join(data_path, filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32) / 255.0
    age_label = int(age)
    return img, age_label



def data_generator(filenames, batch_size):
    while True:
        np.random.shuffle(filenames)
        for i in range(0, len(filenames), batch_size):
            batch_filenames = filenames[i:i+batch_size]
            batch_images = []
            batch_age_labels = []
            for filename in batch_filenames:
                if filename.endswith('.jpg'):
                    img, age_label = load_image(filename)
                    batch_images.append(img)
                    batch_age_labels.append(age_label)
            batch_images = np.array(batch_images)
            batch_age_labels = np.array(batch_age_labels)
            yield batch_images, batch_age_labels

# Define the DL-DL v2 architecture
def create_model():
    model = tf.keras.Sequential()
    
    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    
    # Dense layers
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))  # Output layer for age prediction
    
    return model

# Create an instance of the DL-DL v2 model
model = create_model()

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 26, 26, 128)     

In [ ]:
# Define batch size and number of epochs
batch_size = 128
epochs = 30

# Create data generators for training and validation
train_data_gen = data_generator(train_filenames, batch_size)
val_data_gen = data_generator(val_filenames, batch_size)

# Train the model
model.fit(train_data_gen, steps_per_epoch=len(train_filenames) // batch_size, epochs=epochs, validation_data=val_data_gen, validation_steps=len(val_filenames) // batch_size)


Epoch 1/30
148/148 [==============================] - 53s 340ms/step - loss: 477.2075 - mae: 16.7312 - val_loss: 400.0147 - val_mae: 15.1125
Epoch 2/30
148/148 [==============================] - 52s 344ms/step - loss: 357.7274 - mae: 14.5141 - val_loss: 290.1898 - val_mae: 12.9429
Epoch 3/30
148/148 [==============================] - 50s 337ms/step - loss: 245.6171 - mae: 11.9283 - val_loss: 191.2365 - val_mae: 10.3110
Epoch 4/30
148/148 [==============================] - 50s 337ms/step - loss: 176.6232 - mae: 9.8126 - val_loss: 146.1669 - val_mae: 8.9193
Epoch 5/30
148/148 [==============================] - 50s 338ms/step - loss: 141.8301 - mae: 8.6579 - val_loss: 122.0235 - val_mae: 8.0465
Epoch 6/30
148/148 [==============================] - 49s 330ms/step - loss: 118.5586 - mae: 7.9361 - val_loss: 137.6538 - val_mae: 8.2398
Epoch 7/30
148/148 [==============================] - 48s 326ms/step - loss: 100.4036 - mae: 7.2688 - val_loss: 98.5493 - val_mae: 7.3137
Epoch 8/30
148/148 [==

In [ ]:
model.save('/content/age_model.h5')


In [ ]:
!zip -r /content/age_model.zip /content/age_model

	zip warning: name not matched: /content/age_model

zip error: Nothing to do! (try: zip -r /content/age_model.zip . -i /content/age_model)


In [ ]:
# Evaluate the model on the validation set
val_loss, val_mae = model.evaluate(val_data_gen, steps=len(val_filenames) // batch_size)
print("Validation Loss:", val_loss)
print("Validation MAE:", val_mae)


37/37 [==============================] - 9s 252ms/step - loss: 83.1928 - mae: 6.3576
Validation Loss: 83.19278717041016
Validation MAE: 6.3576340675354
